# Relatório 2

**Nome:** Thiago Lopes <br>
**Matrícula:** 20100358 <br>
**Turma:** T2

# Bibliotecas python utilizadas

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


# Funçoes Auxiliares

In [ ]:
def grafico_mmq(x, y, f, tipo, valor):
    plt.scatter(x, y, label='Pontos', color='red')
    
    x_values = np.linspace(min(x), max(x), 100)
    y_values = [f(x_val) for x_val in x_values]
    
    plt.plot(x_values, y_values, label=tipo)
    
    plt.scatter(valor, f(valor), color='green', label='Valor {}'.format(valor))
    
    plt.xlabel('X')
    plt.ylabel('f(X)')
    
    plt.title('Gráfico MMQ')
    
    plt.legend()
    
    plt.grid(True)
    
    plt.show()
    

# Métodos implementados: 

In [ ]:
def lagrange_interpolation(x, y, z):
    m = len(x)
    Pz = 0

    for i in range(m):
        c = 1
        d = 1
        for j in range(m):
            if i != j:
                c *= (z - x[j])
                d *= (x[i] - x[j])
        Pz += y[i] * (c / d)

    return Pz


In [ ]:
def newton_interpolation(x, y, z):
    m = len(x)
    Dely = y.copy()

    # Construção das diferenças divididas
    for i in range(1, m):
        for k in range(m-1, i-1, -1):
            Dely[k] = (Dely[k] - Dely[k-1]) / (x[k] - x[k-i])

    # Avaliação do polinômio pelo método de Horner
    Pz = Dely[-1]
    for i in range(m-2, -1, -1):
        Pz = Pz * (z - x[i]) + Dely[i]

    return Pz


In [ ]:
#TODO spline natural
def spline_natural():
    return

# Exercícios

## Exercício 1

In [ ]:
# Dados fornecidos
tempos = [0, 10, 30, 60, 90, 120, 140]  # tempo em minutos
distancias = [0.00, 8.00, 27.00, 58.00, 100.00, 145.00, 160.00]  # distância percorrida em km

# Usando os quatro primeiros pontos para calcular a distância percorrida em 45 minutos
tempos_4 = [0, 10, 30, 60]
distancias_4 = [0.00, 8.00, 27.00, 58.00]

# -----------------------------
# lagrange_interpolation
# -----------------------------
print("Lagrange Interpolation")
# A)
distancia_45_min = lagrange_interpolation(tempos_4, distancias_4, 45)
print(f"Distância percorrida nos primeiros 45 minutos: {distancia_45_min:.2f} km")

# B)
tempo_para_metade = lagrange_interpolation(distancias, tempos, 80)
print(f"Tempo necessário para percorrer metade do caminho (80 km): {tempo_para_metade:.2f} minutos")

print("\n")

# -----------------------------
# newton_interpolation
# -----------------------------
print("Newton Interpolation")
# A)
distancia_45_min = newton_interpolation(tempos_4, distancias_4, 45)
print(f"Distância percorrida nos primeiros 45 minutos: {distancia_45_min:.2f} km")

# B)
tempo_para_metade = newton_interpolation(distancias, tempos, 80)
print(f"Tempo necessário para percorrer metade do caminho (80 km): {tempo_para_metade:.2f} minutos")

## Exercício 2

In [ ]:
# Dados fornecidos
comprimentos = [500, 1000, 1500, 2000, 2500, 3000, 3500, 4000]
resistencias = [2.74, 5.48, 7.90, 11.00, 13.93, 16.43, 20.24, 23.52]

# -----------------------------
# lagrange_interpolation
# -----------------------------
print("Lagrange Interpolation")
# Interpolação de grau 2 para 1730 m
comprimentos_grau_2 = [1500, 2000, 2500]
resistencias_grau_2 = [7.90, 11.00, 13.93]
resistencia_1730_g2 = lagrange_interpolation(comprimentos_grau_2, resistencias_grau_2, 1730)
print(f"Resistência para 1730 m (grau 2): {resistencia_1730_g2:.2f} Ohms")

# Interpolação de grau 2 para 3200 m
comprimentos_grau_2 = [3000, 3500, 4000]
resistencias_grau_2 = [16.43, 20.24, 23.52]
resistencia_3200_g2 = lagrange_interpolation(comprimentos_grau_2, resistencias_grau_2, 3200)
print(f"Resistência para 3200 m (grau 2): {resistencia_3200_g2:.2f} Ohms")

# Interpolação de grau 3 para 1730 m
comprimentos_grau_3 = [1000, 1500, 2000, 2500]
resistencias_grau_3 = [5.48, 7.90, 11.00, 13.93]
resistencia_1730_g3 = lagrange_interpolation(comprimentos_grau_3, resistencias_grau_3, 1730)
print(f"Resistência para 1730 m (grau 3): {resistencia_1730_g3:.2f} Ohms")

# Interpolação de grau 3 para 3200 m
comprimentos_grau_3 = [2500, 3000, 3500, 4000]
resistencias_grau_3 = [13.93, 16.43, 20.24, 23.52]
resistencia_3200_g3 = lagrange_interpolation(comprimentos_grau_3, resistencias_grau_3, 3200)
print(f"Resistência para 3200 m (grau 3): {resistencia_3200_g3:.2f} Ohms")

print("\n")

# -----------------------------
# newton_interpolation
# -----------------------------
print("Newton Interpolation")
# Interpolação de grau 2 para 1730 m
comprimentos_grau_2 = [1500, 2000, 2500]
resistencias_grau_2 = [7.90, 11.00, 13.93]
resistencia_1730_g2 = newton_interpolation(comprimentos_grau_2, resistencias_grau_2, 1730)
print(f"Resistência para 1730 m (grau 2): {resistencia_1730_g2:.2f} Ohms")

# Interpolação de grau 2 para 3200 m
comprimentos_grau_2 = [3000, 3500, 4000]
resistencias_grau_2 = [16.43, 20.24, 23.52]
resistencia_3200_g2 = newton_interpolation(comprimentos_grau_2, resistencias_grau_2, 3200)
print(f"Resistência para 3200 m (grau 2): {resistencia_3200_g2:.2f} Ohms")

# Interpolação de grau 3 para 1730 m
comprimentos_grau_3 = [1000, 1500, 2000, 2500]
resistencias_grau_3 = [5.48, 7.90, 11.00, 13.93]
resistencia_1730_g3 = newton_interpolation(comprimentos_grau_3, resistencias_grau_3, 1730)
print(f"Resistência para 1730 m (grau 3): {resistencia_1730_g3:.2f} Ohms")

# Interpolação de grau 3 para 3200 m
comprimentos_grau_3 = [2500, 3000, 3500, 4000]
resistencias_grau_3 = [13.93, 16.43, 20.24, 23.52]
resistencia_3200_g3 = newton_interpolation(comprimentos_grau_3, resistencias_grau_3, 3200)
print(f"Resistência para 3200 m (grau 3): {resistencia_3200_g3:.2f} Ohms")


## Exercício 3

## Exercício 4

4) O número de bactérias, por unidade de volume, existente em uma cultura após x horas é apresentado na tabela:
numero de horas (x) 0 1 2 3 4 5 6
numero de bactérias por
vol. unitário (y) 32 47 65 92 132 190 275
a) Trace o diagrama de dispersão dos dados
b) Use o método dos mínimos quadrados para ajustar os dados as curvas y = ab^x e y = ax^b
c) Verifique e justifique qual a equação do melhor ajuste. Utilize ela para prever o numero de bactérias por vol. unitário (y) em numero de horas (x) igual a 7

In [ ]:
import numpy as np
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score

# Dados fornecidos
x_data = np.array([0, 1, 2, 3, 4, 5, 6])
y_data = np.array([32, 47, 65, 92, 132, 190, 275])

# Funções para ajuste
def func_exp(x, a, b):
    return a * np.power(b, x)

def func_pow(x, a, b):
    return a * np.power(x, b)

# Ajuste para y = ab^x
popt_exp, pcov_exp = curve_fit(func_exp, x_data, y_data, maxfev=10000)
a_exp, b_exp = popt_exp

# Ajuste para y = ax^b (ignorando x=0, pois log(0) é indefinido)
x_data_pow = x_data[1:]
y_data_pow = y_data[1:]
popt_pow, pcov_pow = curve_fit(func_pow, x_data_pow, y_data_pow, maxfev=10000)
a_pow, b_pow = popt_pow

# Previsão para x = 7 horas
x_pred = 7
y_pred_exp = func_exp(x_pred, a_exp, b_exp)
y_pred_pow = func_pow(x_pred, a_pow, b_pow)

# Cálculo do coeficiente de determinação R^2
y_pred_exp_all = func_exp(x_data, a_exp, b_exp)
y_pred_pow_all = func_pow(x_data_pow, a_pow, b_pow)
r2_exp = r2_score(y_data, y_pred_exp_all)
r2_pow = r2_score(y_data_pow, y_pred_pow_all)

# Exibição dos resultados
print("Coeficientes obtidos:")
print("Curva y = ab^x:")
print(f"a = {a_exp}, b = {b_exp}, R^2 = {r2_exp}")

print("\nCurva y = ax^b:")
print(f"a = {a_pow}, b = {b_pow}, R^2 = {r2_pow}")

# Escolha da melhor curva com base no R^2
if r2_exp > r2_pow:
    y_pred = y_pred_exp
    equation = "y = ab^x"
else:
    y_pred = y_pred_pow
    equation = "y = ax^b"

print(f"\nPrevisão para x = 7 horas usando a equação {equation}:")
print(f"y = {y_pred}")


In [17]:
import numpy as np

# Dados fornecidos
x_data = np.array([0, 1, 2, 3, 4, 5, 6])
y_data = np.array([32, 47, 65, 92, 132, 190, 275])

def ajuste_minimos_quadrados(x, y):
    def ajuste_exp(x, y):
        n = len(x)
        sum_x = np.sum(x)
        sum_y = np.sum(np.log(y))
        sum_xx = np.sum(x * x)
        sum_xy = np.sum(x * np.log(y))

        b = (n * sum_xy - sum_x * sum_y) / (n * sum_xx - sum_x ** 2)
        a = np.exp((sum_y - b * sum_x) / n)

        return a, np.exp(b)

    def ajuste_pot(x, y):
        n = len(x)
        sum_log_x = np.sum(np.log(x))
        sum_log_y = np.sum(np.log(y))
        sum_log_xx = np.sum(np.log(x) ** 2)
        sum_log_xy = np.sum(np.log(x) * np.log(y))

        b = (n * sum_log_xy - sum_log_x * sum_log_y) / (n * sum_log_xx - sum_log_x ** 2)
        a = np.exp((sum_log_y - b * sum_log_x) / n)

        return a, b

    # Ajuste exponencial
    a_exp, b_exp = ajuste_exp(x, y)

    # Ajuste de potência (ignorando x=0, pois log(0) é indefinido)
    x_pow = x[1:]
    y_pow = y[1:]
    a_pow, b_pow = ajuste_pot(x_pow, y_pow)

    # Função para prever y para um valor específico de x
    def prever(x_pred, a, b, tipo):
        if tipo == 'exp':
            return a * np.power(b, x_pred)
        elif tipo == 'pot':
            return a * np.power(x_pred, b)
        else:
            raise ValueError("Tipo de ajuste não reconhecido. Use 'exp' para exponencial ou 'pot' para potência.")

    # Previsão para x = 7 horas
    x_pred = 7
    y_pred_exp = prever(x_pred, a_exp, b_exp, 'exp')
    y_pred_pow = prever(x_pred, a_pow, b_pow, 'pot')

    return {
        'a_exp': a_exp,
        'b_exp': b_exp,
        'a_pow': a_pow,
        'b_pow': b_pow,
        'y_pred_exp': y_pred_exp,
        'y_pred_pow': y_pred_pow
    }

# Chamando a função e exibindo os resultados
resultados = ajuste_minimos_quadrados(x_data, y_data)
print("Coeficientes obtidos:")
print("Curva y = ab^x:")
print(f"a = {resultados['a_exp']}, b = {resultados['b_exp']}")

print("\nCurva y = ax^b:")
print(f"a = {resultados['a_pow']}, b = {resultados['b_pow']}")

print("\nPrevisão para x = 7 horas:")
print(f"y = {resultados['y_pred_exp']} (usando a equação y = ab^x)")
print(f"y = {resultados['y_pred_pow']} (usando a equação y = ax^b)")


Coeficientes obtidos:
Curva y = ab^x:
a = 32.1468513163175, b = 1.4269588061798035

Curva y = ax^b:
a = 38.8387110601799, b = 0.9630907218717832

Previsão para x = 7 horas:
y = 387.2741356647424 (usando a equação y = ab^x)
y = 253.02934423935858 (usando a equação y = ax^b)
